In [1]:
from langchain_chroma import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
import json
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

persist_directory = "/Users/mariaborca/Documents/AI_2023-2024/Semestrul 4/Machine Learning/Know-Your-Rights/chroma_db_articles"
vector_store = Chroma(
    collection_name="know_your_rights",
    embedding_function=OpenAIEmbeddings(api_key=OPENAI_API_KEY),
    persist_directory=persist_directory
)

def retrive_relevant_documents(query: str, vector_store: Chroma, k: int = 5):
    results = vector_store.similarity_search_with_score(query, k=k)
    results.sort(key=lambda x: x[1], reverse=False)
    return results

def get_openai_llm(model: str = "gpt-4o-mini"):
    return OpenAI(
        model=model,
        temperature=0,
        max_tokens=1000,
        api_key=OPENAI_API_KEY)

def get_local_model(model: str = "phi-4"):
    base_url = "http://localhost:1234/v1"
    api_key = "lm-studio"

    return ChatOpenAI(
        base_url=base_url,
        api_key=api_key,
        temperature=0.9,
        model=model
    )

def get_response(query, llm, template=None, context=None):
    if template is None:
        template = """
        Folosește **doar informațiile din contextul de mai jos** pentru a răspunde la întrebare. 
        Dacă răspunsul nu este prezent în context, răspunde cu „Nu știu”.

        Context:
        {context}

        Întrebare:
        {question}

        Răspuns:

        """
        
    prompt = template.format(context=context, question=query)
    response = llm.invoke(prompt)
    return response.content if type(response)!=str else llm.invoke(prompt)

def load_test_cases(file_path):
    with open(file_path, 'r') as file:
        test_cases = json.load(file)
    return test_cases

In [11]:
from deepeval import evaluate
from deepeval.metrics import GEval, AnswerRelevancyMetric, ContextualRecallMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.evaluate.configs import DisplayConfig

import io
import contextlib

def extract_metric_info(evaluation_result):
    results = []
    for test_result in evaluation_result.test_results:
        for metric_data in test_result.metrics_data:
            results.append({
                "metric": metric_data.name,
                "success": metric_data.success,
                "score": metric_data.score,
                "reason": metric_data.reason
            })
    return results

silent_display_config = DisplayConfig(
    print_results=False,
    show_indicator=False
)

def test_answer_relevancy(user_query, actual_answer):
    metric = AnswerRelevancyMetric(
        threshold=0.7,
        include_reason=True,
        verbose_mode=False,
    )

    test_case = LLMTestCase(
        input=user_query,
        actual_output=actual_answer)
    
    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def test_contextual_recall(user_query, expected_answer, actual_answer, retrival_context):
    metric = ContextualRecallMetric(
        threshold=0.7,
        include_reason=True,
        verbose_mode=False
    )
    test_case = LLMTestCase(
        input=user_query,
        expected_output=expected_answer,
        actual_output=actual_answer,
        retrieval_context=retrival_context
    )

    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def test_faithfulness(user_query, actual_answer, retrival_context):
    metric = FaithfulnessMetric(
        threshold=0.7,
        verbose_mode=False,
        model="gpt-4o-mini",
    )
    test_case = LLMTestCase(
        input=user_query,
        actual_output=actual_answer,
        retrieval_context=retrival_context
    )
    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def test_understanding(user_query, expected_answer, actual_answer):
    metric = GEval(
        name="Understanding",
        evaluation_steps=(
            ["Răspunsul este formulat într-un mod clar și accesibil pentru o persoană fără cunoștințe juridice? ",
            "Poate utilizatorul să înțeleagă ce are voie să facă sau ce trebuie să facă, pe baza acestui răspuns?"]
        ),
        threshold=0.7,
        evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
        verbose_mode=False,
        model="gpt-4o-mini",
    )
    test_case = LLMTestCase(
        input=user_query,
        expected_output=expected_answer,
        actual_output=actual_answer
    )
    return evaluate(
        test_cases=[test_case],
        metrics=[metric],
        display_config=silent_display_config
    )

def run_evaluation(test_cases, vector_store, llm, prompt):
    results = []
    for test_case in test_cases:
        user_query = test_case['question']
        expected_answer = test_case['expected_answer']

        retrival_context = retrive_relevant_documents(user_query, vector_store, k=3)
        retrival_context = [doc.page_content for doc, _ in retrival_context]
        context = "\n\n".join(retrival_context)
        
        # Run RAG to get the actual answer
        actual_answer = get_response(user_query, llm, prompt, context)

        # Evaluate the answers
        relevancy_result = test_answer_relevancy(user_query, actual_answer)
        recall_result = test_contextual_recall(user_query, expected_answer, actual_answer, retrival_context)
        faithfulness_result = test_faithfulness(user_query, actual_answer, retrival_context)
        understanding_result = test_understanding(user_query, expected_answer, actual_answer)

        relevancy_result = extract_metric_info(relevancy_result)
        recall_result = extract_metric_info(recall_result)
        faithfulness_result = extract_metric_info(faithfulness_result)
        understanding_result = extract_metric_info(understanding_result)

        print(f"""
            Query: {user_query}
            Expected: {expected_answer}
            Actual: {actual_answer}
            Relevancy Score: {relevancy_result}
            Recall Score: {recall_result}
            Faithfulness Score: {faithfulness_result}
            Understanding Score: {understanding_result}
            """)
       

        results.append({
            'user_query': user_query,
            'expected_answer': expected_answer,
            'actual_answer': actual_answer,
            'relevancy': relevancy_result,
            'recall': recall_result,
            'faithfulness': faithfulness_result,
            'understanding': understanding_result,
        })
    
    return results

def start_testing(models_list, test_cases, prompt):
    results_for_models = {model : [] for model in models_list}
    for test in test_cases:
        print(f"Running tests for query: {test['question']}")
        for model in models_list:
            print(f"Using model: {model}")
            llm = get_local_model(model=model)
            results = run_evaluation([test], vector_store, llm, prompt)
            results_for_models[model].append(results)
        print(f"Finished tests for model: {model}\n")
        print("=" * 50)
        print("\n")
    return results_for_models


    

In [ ]:
models_list = ["mistral-7b-instruct-v0.1", "rollama3-8b-instruct", "rollama2-7b-instruct"]

prompt = """
Ești un asistent juridic virtual, specializat în explicarea drepturilor cetățenilor într-un limbaj simplu și accesibil. Răspunzi în limba română, folosind informațiile din documente juridice oficiale precum Constituția României, Codul Muncii sau alte acte normative relevante.

Când răspunzi, urmează aceste reguli:
 - Explică termenii juridici într-un mod clar, ca și cum ai vorbi cu cineva fără pregătire juridică.
 - Fii concis, dar oferă informații corecte și complete.
 - Dacă este cazul, poți cita articole de lege (ex: „Conform articolului X din Codul Muncii...”).
 - Nu inventa informații. Dacă întrebarea nu are un răspuns clar în documentele juridice disponibile, spune că nu poți oferi un răspuns sigur.
 - Nu oferi sfaturi legale personalizate; explică doar cadrul legal general.

Întrebarea utilizatorului este: {question}

Folosește următoarele informații extrase din documentele juridice:
{context}

Răspunsul tău trebuie să fie în limba română, clar, politicos și ușor de înțeles.
"""

start_testing(models_list, tests_list, prompt)

Running tests for query: Cât timp poate fi reținută o persoană înainte de a fi arestată?
Using model: mistral-7b-instruct-v0.1


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


            Query: Care sunt drepturile și obligațiile părților în cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului?
            Expected: În cazul concedierii pentru motive care nu țin de persoana salariatului, angajatorul trebuie să respecte proceduri stricte și să asigure drepturi specifice salariatului. Concedierea este permisă doar pentru desființarea locului de muncă din motive economice, tehnologice, structurale sau similare. Angajatorul are obligația să acorde un preaviz de minimum 20 de zile lucrătoare și să respecte criterii obiective de selecție (performanțe profesionale, vechime, sarcini de întreținere).
            Actual: 
În cazul unei concedieri individuale pentru motive care nu țin de persoana salariatului, drepturile și obligațiile părților sunt după cum urmează:

1. Salariatul are dreptul la protecție adecvată împotriva concedierii nelegitime (motive care nu țin de persoana sa).
2. Angajatorul trebuie să demonstreze că desființare

### **Summary of Romanian Prompt Testing for Three Models**

I tested three language models — **Rollama3-8b**, **Rollama2-7b**, and **Mistral** — on six legal questions in Romanian. Each model’s responses were evaluated based on four key aspects: **relevance** (if the answer matches the question), **recall** (if it uses the correct information), **faithfulness** (if it’s factually accurate), and **understanding** (if it’s easy to read).

#### ✅ Rollama3-8b

* This model had the **best results overall**.
* **4 out of 6 answers** passed at least two evaluation tests.
* The answers were **grammatically correct** and mostly accurate.
* However, some responses were **too long and complicated**, which made them harder to understand.

#### ⚠️ Rollama2-7b

* Performance was **average**.
* It gave **3 strong answers** out of 6.
* The language was sometimes **easier to read** than Rollama3, but there were **grammar mistakes** and **some incorrect or missing legal facts**.
* Still, it could be a decent alternative depending on the use case.

#### ❌ Mistral

* This model had the **worst performance**.
* Although **4 answers passed** two tests, **one answer failed all tests**.
* The grammar was **poor**, and many sentences were **confusing or incorrect**.
* The output often included **nonsense words**, **contradictions**, or **legal errors**.

I **expected Mistral to perform poorly** since it wasn’t trained on Romanian data. Because of this and its low quality, I’ve decided to **drop Mistral** from my testing process. This will also help make the prompt testing process **faster and more focused**.

### **Summary of Prompt Changes and Their Purpose**

You updated your prompt to improve the quality and reliability of answers in Romanian, especially when testing models like Rollama and Mistral. Below are the main changes you made and why:

---

#### 1. **Changed role from “asistent juridic” to “ghid juridic”**

* **Before**: “Ești un asistent juridic virtual...”
* **Now**: “Ești un ghid juridic virtual...”
* **Reason**: "Ghid" sounds more accessible and avoids confusion with professional legal advisors. It better reflects the goal of explaining the law in simple terms.

---

#### 2. **Removed the option to cite legal articles**

* **Before**: “Dacă este cazul, poți cita articole de lege...”
* **Now**: “Nu cita articole de lege și nu inventa numere de articole.”
* **Reason**: Many models (especially Mistral) hallucinate article numbers. Removing this prevents factual errors and misleading answers.

---

#### 3. **Clear fallback when information is missing**

* **Before**: “...spune că nu poți oferi un răspuns sigur.”
* **Now**: “...răspunde cu: „Nu am putut genera un răspuns.””
* **Reason**: You wanted a **consistent and predictable fallback** when the context is unclear or insufficient, instead of vague answers.

---

#### 4. **Added guidance for short, clear phrasing**

* **New rule**: “Scrie răspunsul în propoziții scurte și clare. Poți folosi listă cu puncte, dacă este nevoie.”
* **Reason**: Some responses from the models were long and hard to understand. This rule improves clarity and readability.

---

### ✅ **Overall Goal of the Changes**

These updates help the model:

* stay grounded in the provided context,
* avoid hallucinations,
* use simple, readable language,
* and fail gracefully when unsure.

This makes the testing process **more reliable** and results easier to evaluate.


In [18]:
models_list = ["rollama3-8b-instruct", "rollama2-7b-instruct"]

prompt_v2 = """
Ești un ghid juridic virtual, care ajută cetățenii să înțeleagă legea pe înțelesul tuturor. 

Când răspunzi, urmează aceste reguli:
 - Explică termenii juridici într-un mod clar, ca și cum ai vorbi cu cineva fără pregătire juridică.
 - Fii concis, dar oferă informații corecte și complete.
 - Nu cita articole de lege și nu inventa numere de articole.
 - Scrie răspunsul în propoziții scurte și clare. Poți folosi listă cu puncte, dacă este nevoie.
 - Nu inventa informații. Dacă informația nu se regăsește clar în contextul oferit, răspunde cu: **„Nu am putut genera un răspuns.”**
 - Nu oferi sfaturi legale personalizate; explică doar cadrul legal general.

Întrebarea utilizatorului este: {question}

Folosește următoarele informații extrase din documentele juridice:
{context}

Răspunsul tău trebuie să fie în limba română, clar, politicos și ușor de înțeles.
"""

model_results = start_testing(models_list, tests_list, prompt_v2)


Running tests for query: Cât timp poate fi reținută o persoană înainte de a fi arestată?
Using model: rollama3-8b-instruct


✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

...

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.


            Query: Care este vârsta legală minimă pentru încheierea unei căsătorii în România?
            Expected: Căsătoria se poate încheia dacă viitorii soți au împlinit vârsta de 18 ani.
            Actual: Vârsta legală minimă pentru încheierea unei căsătorii în România este de 16 ani, în cazul în care ambii viitoare soți au împlinit vârsta acestui vârstă. Cu toate acestea, nulitatea poate fi acoperită dacă până la hotărârea definitivă a instanței, ambele soții îndeplinesc vârsta de 18 ani sau dacă o soție este gravidă.
            Relevancy Score: [{'metric': 'Answer Relevancy', 'success': True, 'score': 1.0, 'reason': 'The score is 1.00 because the response perfectly addresses the question about the legal minimum age for marriage in Romania without any irrelevant information. Great job!'}]
            Recall Score: [{'metric': 'Contextual Recall', 'success': True, 'score': 1.0, 'reason': 'The score is 1.00 because the expected output perfectly matches the information from the

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

KeyboardInterrupt: 

Error updating test run to disk: [Errno 2] No such file or directory: 
'.deepeval/.temp_test_run_data.json'


In [16]:
from statistics import mean, stdev
from collections import defaultdict

def analyze_model_results(model_results):
    """
    Input: model_results - list (or list of lists) of dictionaries with keys:
        'user_query', 'expected_answer', 'actual_answer', 'relevancy', 'recall', 'faithfulness', 'understanding'

    Output:
        - average score per metric
        - pass rate per metric
        - a unified list of examples (one per test case, includes all metrics)
    """

    # Flatten if it's a list of lists
    if any(isinstance(item, list) for item in model_results):
        flat_results = [entry for sublist in model_results for entry in sublist]
    else:
        flat_results = model_results

    metric_data = defaultdict(lambda: {
        "scores": [],
        "successes": 0,
        "total": 0
    })

    unified_examples = []

    for result in flat_results:
        test_example = {
            "query": result["user_query"],
            "expected": result["expected_answer"],
            "actual": result["actual_answer"],
            "metrics": {}
        }

        for metric_name in ["relevancy", "recall", "faithfulness", "understanding"]:
            metric_entries = result.get(metric_name, [])
            for entry in metric_entries:
                name = entry["metric"]
                score = entry["score"]
                success = entry["success"]
                reason = entry.get("reason", "")

                # Track overall stats
                metric_data[name]["scores"].append(score)
                metric_data[name]["total"] += 1
                if success:
                    metric_data[name]["successes"] += 1

                # Store this metric result under the test case
                test_example["metrics"][name] = {
                    "score": score,
                    "success": success,
                    "reason": reason
                }

        unified_examples.append(test_example)

    summary = {}
    for metric, data in metric_data.items():
        scores = data["scores"]
        summary[metric] = {
            "avg_score": round(mean(scores), 3) if scores else None,
            "std_dev": round(stdev(scores), 3) if len(scores) > 1 else 0.0,
            "pass_rate": f"{round((data['successes'] / data['total']) * 100, 1)}%" if data["total"] > 0 else "N/A",
            "total_cases": data["total"]
        }

    # Add unified examples only once
    summary["examples"] = unified_examples

    return summary


In [17]:
for model_name, model_result_list in model_results.items():
    print(f"Evaluating model: {model_name}")
    results_summary = analyze_model_results(model_result_list)
    print(json.dumps(results_summary, indent=4, ensure_ascii=False))

# print(model_results["rollama3-8b-instruct"])


Evaluating model: rollama3-8b-instruct
{
    "Answer Relevancy": {
        "avg_score": 0.715,
        "std_dev": 0.358,
        "pass_rate": "50.0%",
        "total_cases": 6
    },
    "Contextual Recall": {
        "avg_score": 0.733,
        "std_dev": 0.313,
        "pass_rate": "50.0%",
        "total_cases": 6
    },
    "Faithfulness": {
        "avg_score": 0.578,
        "std_dev": 0.357,
        "pass_rate": "33.3%",
        "total_cases": 6
    },
    "Understanding (GEval)": {
        "avg_score": 0.5,
        "std_dev": 0.16,
        "pass_rate": "16.7%",
        "total_cases": 6
    },
    "examples": [
        {
            "query": "Cât timp poate fi reținută o persoană înainte de a fi arestată?",
            "expected": "Reținerea nu poate depăși 24 de ore. După acest termen, persoana trebuie eliberată sau arestată preventiv prin decizia unui judecător.",
            "actual": "Potrivit Legii privind acțiunile și măsurile penale (Legea nr. 281/2005), reținerea poate d

### 🔍 **Summary of Results: Rollama3-8b vs Rollama2-7b (with New Prompt)**

#### ✅ **Rollama3-8b**

* **Scores**: 50% relevance, 50% recall, 33% faithfulness, 17% understanding
* **Grammar**: Generally correct, but the style is overly legal and hard to follow.
* **Issues**: Answers are often too formal, with long sentences and complex structures. Sometimes includes off-topic info.

#### ✅ **Rollama2-7b**

* **Scores**: 67% relevance, 50% recall, 50% faithfulness, **0% understanding**
* **Grammar**: Acceptable, but sentences are long and unclear.
* **Issues**: Provides lots of details, but they’re hard to process. No answers were considered easy to understand.

---

### ⚖️ **Conclusion**

* **Rollama2-7b** gave more accurate and relevant answers overall.
* **Rollama3-8b** was better grammatically, but less faithful and harder to understand.
* Neither model fully achieved the goal of clear, simple legal explanations.

---


In [ ]:
prompt_v3 = """ 
Ești un ghid juridic virtual care explică legea în mod clar și accesibil pentru orice cetățean, fără termeni complicați sau ambigui.

Când răspunzi, respectă aceste reguli:
 - Folosește propoziții clare, scurte și bine structurate (nu mai mult de 2 fraze per idee).
 - Explică termenii juridici pe înțelesul oricui, fără a presupune cunoștințe legale.
 - Nu menționa articole de lege și nu inventa surse legale.
 - Nu adăuga informații care nu apar în contextul oferit.
 - Dacă contextul nu oferă un răspuns clar, scrie simplu: **„Nu am putut genera un răspuns.”**
 - Evită limbajul prea tehnic sau abstract. Fii prietenos, dar neutru.

Întrebarea utilizatorului este:  
{question}

Informațiile disponibile sunt:  
{context}

Scrie răspunsul în limba română. Acesta trebuie să fie clar, politicos și ușor de înțeles.
"""
model_results = start_testing(models_list, tests_list, prompt_v3)
